In [1]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [2]:
pip -q install langchain langchain-community langchain_groq sentence-transformers langchain-huggingface huggingface-hub faiss-gpu

### USING PICKLE FILE FOR CHAT HISTORY MANAGEMENT

In [3]:
model_name ='llama3-8b-8192'
groq_key ='key'

In [4]:
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import LLMChain, RetrievalQA, ConversationChain
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables import ConfigurableFieldSpec
import pickle

In [5]:
llm = ChatGroq(model_name=model_name, groq_api_key=groq_key, temperature=0.5, max_tokens=256)

In [6]:
session_id = 'sundeep_0'
username = 'sun00009'

In [43]:
store = {}

In [44]:
def get_session_history(user_id: str, conversation_id: str) -> BaseChatMessageHistory:
    if (user_id, conversation_id) not in store:
        store[(user_id, conversation_id)] = ChatMessageHistory()
    return store[(user_id, conversation_id)]

In [45]:
template = [
        (
            "system",
            "You're a helpful converastion chatbot. Keep your response to limit of 100 words only. Utilize the context if needed {context}",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]

In [46]:
prompt = ChatPromptTemplate.from_messages(template)

In [ ]:
model_llm = prompt | llm

In [12]:
conversation = RunnableWithMessageHistory(
    runnable=model_llm,
    get_session_history=get_session_history,
    input_messages_key='input',
    history_messages_key='chat_history',
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name=username,
            description="Unique identifier for the user.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="conversation_id",
            annotation=str,
            name=session_id,
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        ),
    ],)

In [13]:
conversation.invoke({'input':input()},
                    config={"configurable": {"user_id": username, 'conversation_id': session_id}}).content

Hi


"Hi! It's nice to meet you. Is there something I can help you with or would you like to chat? I'm here to listen and assist with any questions or topics you'd like to discuss."

In [14]:
conversation.invoke({'input':input()},
                    config={"configurable": {"user_id": username, 'conversation_id': session_id}}).content

Hi


"Hi again! How's your day going so far?"

In [15]:
conversation.invoke({'input':input()},
                    config={"configurable": {"user_id": username, 'conversation_id': session_id}}).content

Hi


"Hi again! It seems like you're in a chatty mood! What's on your mind? Want to talk about something specific or just shoot the breeze? I'm all ears!"

In [16]:
store[username,session_id]

InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi'), AIMessage(content="Hi! It's nice to meet you. Is there something I can help you with or would you like to chat? I'm here to listen and assist with any questions or topics you'd like to discuss.", response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 37, 'total_tokens': 80, 'completion_time': 0.03322573, 'prompt_time': 0.007854176, 'queue_time': None, 'total_time': 0.041079906}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_af05557ca2', 'finish_reason': 'stop', 'logprobs': None}, id='run-fbb41cf5-c20b-47b1-ae58-63f0cda360fb-0', usage_metadata={'input_tokens': 37, 'output_tokens': 43, 'total_tokens': 80}), HumanMessage(content='Hi'), AIMessage(content="Hi again! How's your day going so far?", response_metadata={'token_usage': {'completion_tokens': 12, 'prompt_tokens': 90, 'total_tokens': 102, 'completion_time': 0.009073979, 'prompt_time': 0.019118537, 'queue_time': None, 'total_time': 0.02

In [17]:
# with open('chat_history.pkl', 'wb') as f:
#     pickle.dump(store, f)

# with open('chat_history.pkl', 'rb') as f:
#     store = pickle.load(f)

### ADDING RAG

In [61]:
from langchain.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import DataFrameLoader
from langchain.schema.vectorstore import VectorStoreRetriever
import pandas as pd
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.chains import create_history_aware_retriever
import huggingface_hub
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

In [13]:
huggingface_hub.login(token='key')

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [14]:
df = pd.read_csv('docs.txt')

In [15]:
text = DataFrameLoader(data_frame=df,page_content_column='page_content')
documents = text.load()

In [16]:
docs = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0).split_documents(documents)

In [17]:
embeddings = HuggingFaceEmbeddings(model_name='BAAI/bge-m3')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [18]:
db = FAISS.from_documents(documents=docs, embedding=embeddings)

In [19]:
# For redis
# https://python.langchain.com/v0.2/docs/integrations/vectorstores/redis/

In [20]:
retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":4,"fetch_k":10})

In [60]:
context = itemgetter("input") | retriever

In [62]:
first_step = RunnablePassthrough.assign(context=context)

In [63]:
model_llm = first_step | prompt | llm

In [64]:
rag_template = [
        (
            "system",
            "Given a chat history and the latest user question "
            "which might reference context in the chat history, "
            "formulate a standalone question which can be understood "
            "without the chat history. Do NOT answer the question, "
            "just reformulate it if needed and otherwise return it as is.",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]

In [65]:
rag_prompt = ChatPromptTemplate.from_messages(rag_template)

In [66]:
history_aware_retriever = create_history_aware_retriever(
    llm = model_llm,
    retriever = retriever,
    prompt = rag_prompt)

In [67]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

In [68]:
question_answer_chain = create_stuff_documents_chain(llm=model_llm, prompt=prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [69]:
conversation_rag_chain = RunnableWithMessageHistory(
    runnable=model_llm,
    rag_chain=rag_chain,
    get_session_history=get_session_history,
    input_messages_key='input',
    history_messages_key='chat_history',
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name=username,
            description="Unique identifier for the user.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="conversation_id",
            annotation=str,
            name=session_id,
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        ),
    ],)

In [70]:
conversation_rag_chain.invoke({'input':input()},
                    config={"configurable": {"user_id": username, 'conversation_id': session_id}}).content

Hi


"Hi! It's nice to meet you. Is there something I can help you with or would you like to chat about something in particular?"

In [71]:
conversation_rag_chain.invoke({'input':input()},
                    config={"configurable": {"user_id": username, 'conversation_id': session_id}}).content

Get me some beer


"I'm just a chatbot, I don't have the ability to physically get you a beer. But I can certainly chat with you about it! What kind of beer are you in the mood for?"